In [11]:
run_cycle = 1

In [12]:
import random
import datetime
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.app.random_request import RandomRequestApp

import numpy as np

import csv
import os

import sequence.utils.log as log

def write_to_csv(file_path, data):
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

now = datetime.datetime.now()
date_time_str =now.strftime('%Y%m%d_%H%M%S')

csv_file_path = "/home/uom/howe0427/SeQUeNCe/example/generated_data/generated_data_5node_100km_alltracked4.csv"
print(f"File exists before opening: {os.path.isfile(csv_file_path)}")
print(f"File size before opening: {os.path.getsize(csv_file_path) if os.path.exists(csv_file_path) else 'Not exists'}")

# Initialize CSV file once
def initialize_csv(file_path, headers):
    if not os.path.isfile(file_path) or os.path.getsize(file_path) == 0:
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(headers)

initialize_csv(csv_file_path, ['Router Name', 'Node Seed', 'Entanglement Time (ps)', 'Fidelity'])


def test(sim_time, cc_delay, qc_atten, qc_dist, num_nodes, starting_node, desired_node):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ms)
    qc_atten: attenuation on quantum channels (dB/m)
    qc_dist: distance of quantum channels (km) 
    num_nodes: number of router nodes you want the system to have
    """
    
    PS_PER_MS = 1e9
    M_PER_KM = 1e3
    
    qc_dist = qc_dist.astype(float)
    # convert units for cc delay (to ps) and qc distance (to m)
    cc_delay *= PS_PER_MS
    qc_dist *= M_PER_KM
    
    raw_fidelity = 0.75
    
    log_filename = "multi_node_framework_log " + str(run_cycle) + ".log"
    
    # construct the simulation timeline; the constructor argument is the simulation time (in ps)
    tl = Timeline(sim_time * PS_PER_MS)
    tl.show_progress = True
    
    #log.set_logger(__name__, tl, log_filename)
    #log.set_logger_level('INFO')
    #log.track_module('timeline')
    #log.track_module('network_manager')
    #log.track_module('generation')
    #log.track_module('swapping')
    #log.track_module('node')
    #log.track_module('resource_manager')
    # DO THIS IN A LOOP, HOW DO THE MEMORIES WORK THOUGH?, 
    # MAKE ROUTERS AN ARRAY AND PASS IN ANOTHER PARAMETER WHICH SPECIFIES NUMBER OF NODES
    routers = [QuantumRouter] * num_nodes
    ## create our quantum network and update parameters as needed
    for i in range(0, num_nodes):
        routers[i] = QuantumRouter("r" + str(i), tl, 100)
        
    BSMNodes = [BSMNode] * (num_nodes - 1)
    for i in range(0, num_nodes - 1):
        BSMNodes[i] = BSMNode("m" + str(i), tl, [routers[i].name, routers[i+1].name])    

    for i in range(0,num_nodes - 1):
        routers[i].add_bsm_node(BSMNodes[i].name, routers[i + 1].name)
        routers[i + 1].add_bsm_node(BSMNodes[i].name, routers[i].name)
    
    # set seeds for random generators
    nodes = []
    nodes_seeds = []
    for router in routers:
        nodes.append(router)
    for bsm_node in BSMNodes:
        nodes.append(bsm_node)
    for i, node in enumerate(nodes):
        print("length of nodes is ", (len(nodes) + 1) / 2)
        #seed_random = random.randint(1, 100)
        seed_random = random.randint(1, (len(nodes) + 1) / 2)
        node.set_seed(seed_random)
        nodes_seeds.append(seed_random)
        #print(node.name,seed_random)
    
    for node in routers:
        memory_array = node.get_components_by_type("MemoryArray")[0]
        # we update the coherence time (measured in seconds) here
        memory_array.update_memory_params("coherence_time", 10)
        # and similarly update the fidelity of entanglement for the memories
        memory_array.update_memory_params("raw_fidelity", raw_fidelity)
        
    SWAP_SUCC_PROB = 0.64
    SWAP_DEGRADATION = 0.99
    for node in routers:
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)
    
    # create all-to-all classical connections
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            # construct a classical communication channel
            # (with arguments for the channel name, timeline, length (in m), and delay (in ps))
            cc = ClassicalChannel("cc_%s_%s"%(node1.name, node2.name), tl, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2.name)
            
    # NOTE: CLASSICAL CHANNEL DELAY CAN NOT CURRENTLY BE DIFFERENT FOR EACH CLASSICAL CHANNEL

    
    # DO THIS IN A LOOP, TAKE IN qc_dist, qc_atten as arrays 
            
    # create quantum channels linking r1 and r2 to m1
    # (with arguments for the channel name, timeline, attenuation (in dB/km), and distance (in m))
    qc_array = [None] * (num_nodes - 1) * 2
    j = 0
    for i in range(0, len(qc_array), 2):
        qc_array[i] = QuantumChannel("qc_r" + str(j) + "_m" + str(j), tl, qc_atten, qc_dist[i])
        qc_array[i + 1] = QuantumChannel("qc_r" + str(j + 1) + "_m" + str(j), tl, qc_atten, qc_dist[i + 1])
        qc_array[i].set_ends(routers[j], BSMNodes[j].name)
        qc_array[i + 1].set_ends(routers[j + 1], BSMNodes[j].name)
        j += 1

    # routing table setup and forwarding rules
    for i in range(len(routers)):
        for j in range(len(routers)):
            if j != i:
                if i > j:
                    routers[i].network_manager.protocol_stack[0].add_forwarding_rule("r" + str(j), "r" + str(i-1))
                    print("router ", i , " adds ", "r" + str(j), "r" + str(i-1))
                if i < j:
                    routers[i].network_manager.protocol_stack[0].add_forwarding_rule("r" + str(j), "r" + str(i+1))
                    print("router ", i , " adds ",  "r" + str(j), "r" + str(i+1))
    print("Protocol stack is: ", len(routers[0].network_manager.protocol_stack))
    
    ## run our simulation
    
    
    apps = []
    router_names = [node.name for node in routers]
    for i, node in enumerate(routers):
        app_node_name = node.name
        others = router_names[:]
        others.remove(app_node_name)
        app = RandomRequestApp(node, others, i,
                               min_dur=1e13, max_dur=2e13, min_size=10,
                               max_size=25, min_fidelity=0.8, max_fidelity=1.0)
        apps.append(app)
        app.start()
    
    tl.init()
    # we use the network manager of an end router to make our entanglement request
    # here, the arguments are:
    # (1) the destination node name,
    # (2) the start time (in ps) of entanglement,
    # (3) the end time (in ps) of entanglement,
    # (4) the number of memories to entangle, and
    # (5) the desired fidelity of entanglement.
    routers[starting_node].network_manager.request("r" + str(desired_node), 1e12, 1e14, 1, 0.9)

    tick = time.time()
    tl.run()
    #print("execution time %.2f sec" % (time.time() - tick))

    routers_to_plot = routers
    
    file_exists = os.path.isfile(csv_file_path) and os.path.getsize(csv_file_path) > 0

    print(f"Router: {router.name}, Memory Manager Length: {len(router.resource_manager.memory_manager)}")
    for info in router.resource_manager.memory_manager:
        print(f"Checking memory info: Entangle Time = {getattr(info, 'entangle_time', 'Not set')}")
        if hasattr(info, 'entangle_time') and info.entangle_time > 0:
            print(f"Writing to CSV: {info}")
    # Open the file for appending, handle it within 'with' to ensure it closes properly
    with open(csv_file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        # Write headers if the file did not exist or was empty
        headers = ['Router Name', 'Node Seed', 'Entanglement Time (ps)', 'Fidelity']

        print("Writing static data to test.")
        writer.writerow(['Static Router', 'Static Seed', 12345.678, 0.99])
        
        if not file_exists:
            writer.writerow(headers)

        # Iterate through selected routers
        for i, router in enumerate(routers_to_plot):
            # Make sure your router objects have a resource_manager and a memory_manager correctly configured
            for info in router.resource_manager.memory_manager:
                # Check if info has an attribute 'entangle_time' and it is greater than zero
                if hasattr(info, 'entangle_time') and info.entangle_time > 0:
                    entangle_time = info.entangle_time / 1e12  # Convert time as necessary
                    fidelity = info.fidelity
                    row_data = [router.name, nodes_seeds[i], entangle_time, fidelity]
                    writer.writerow(row_data)
    #print(tl.run_counter)
    #print(tl.schedule_counter)
    #print(tl.entities)
    #print(tl.events)
#interactive_plot = interact(test, sim_time=(2000, 4000, 500), cc_delay=(0.1, 1, 0.1), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
#interactive_plo

num_nodes = 4
size = 2*(num_nodes - 1)
#qc_distances = np.random.randint(1, 9, size)
tot_distance = 1000
qc_distances = np.ones(size) * (tot_distance / size)
#qc_distances = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
#print(qc_distances)
starting = 0
desired = 3
test(20000, 0.1, 1e-5, qc_distances, num_nodes, starting, desired)

INFO:__main__:Create Node r0
INFO:__main__:Create Node r1
INFO:__main__:Create Node r2
INFO:__main__:Create Node r3
INFO:__main__:Create Node m0
INFO:__main__:Create Node m1
INFO:__main__:Create Node m2


File exists before opening: True
File size before opening: 47967
length of nodes is  4.0
length of nodes is  4.0
length of nodes is  4.0
length of nodes is  4.0
length of nodes is  4.0
length of nodes is  4.0
length of nodes is  4.0
router  0  adds  r1 r1
router  0  adds  r2 r1
router  0  adds  r3 r1
router  1  adds  r0 r0
router  1  adds  r2 r2
router  1  adds  r3 r2
router  2  adds  r0 r1
router  2  adds  r1 r1
router  2  adds  r3 r3
router  3  adds  r0 r2
router  3  adds  r1 r2
router  3  adds  r2 r2
Protocol stack is:  2


NameError: name 'RandomRequestApp' is not defined

### Results

We note that the number of entangled memories increses exponentially over time. This is due to memory expiration, with a coherence time of 0.3 seconds, as well as the consumption of memories for purification and swapping soon after they are entangled. We also see distinct layers of fidelities, reflecting the number of times an individual memory has been purified or swapped.